In [1]:
import numpy as np
from matplotlib import pyplot as plt
import aim
from aim import Run
from tqdm.notebook import tqdm

In [2]:
repo = aim.Repo.from_path('../')

In [ ]:
stats = {
    "class_name": [],
    "model": [],
    "auc_max": [],
    "auc_max_50": [],
    "auc_argmax": [],
    "sample_size": []
}

In [ ]:
def process_auc(run):
    print(run.experiment)
    try:
        class_name, model, _, sample_size, _ = run.experiment.split('_')
    except Exception as e:
        class_name, model, _, _, sample_size, _ = run.experiment.split('_')
    
    for metric in run.metrics():
        if metric.name != 'auc':
            continue
        
        aucs = np.array([x[1][0] for x in metric.data])
        
        stats['class_name'].append(class_name)
        stats['model'].append(model)
        stats['auc_max'].append(aucs.max())
        stats['auc_max_50'].append(aucs[:50].max())
        stats['auc_argmax'].append(aucs.argmax())
        if 'K' in sample_size:
            sample_size = sample_size.replace('K', '000')
        stats['sample_size'].append(int(sample_size))
                
        break

In [4]:
for run_hash in tqdm(repo.list_all_runs()):
    try:
#         if run_hash == 'b330b5ed44d34351b4f44f97':
#             continue
#         print(run_hash)
        run = Run(run_hash)
#         if ("mae_patches" in run.experiment) or ("dino_patches" in run.experiment) or  ("sup_vit_patches" in run.experiment):
#             process_auc(run)
    except Exception as e:
        print(f"{run_hash}", e)

  0%|          | 0/437 [00:00<?, ?it/s]

76af2b6c854b4faa911db542 The file lock '/home/hkhachatrian/mae/.aim/locks/76af2b6c854b4faa911db542.softlock' could not be acquired.
0af5734eccbf415e99e664b8 The file lock '/home/hkhachatrian/mae/.aim/locks/0af5734eccbf415e99e664b8.softlock' could not be acquired.


KeyboardInterrupt: 

In [ ]:
for key in stats:
    stats[key] = np.array(stats[key])

In [ ]:
classes = np.unique(stats['class_name'])

In [ ]:
len(classes)

In [ ]:
plt.figure(figsize=(10,4))
for cls in classes:
    mask = (stats['class_name'] == cls) & (stats['model'] == 'mae')
    x = stats['sample_size'][mask]
    y = stats['auc_max'][mask]
    p = np.argsort(x)
    plt.plot(x[p], y[p], label=cls)
plt.xscale('log')
plt.xlim(80, 40000)
plt.legend()


In [ ]:
  mask = stats['auc_max'] - stats['auc_max_50'] > 0.05

In [ ]:
stats['class_name'][mask], stats['model'][mask], 

In [ ]:
mask = stats['sample_size'] == 10000
plt.figure()
plt.scatter(stats['auc_max_50'], stats['auc_max'])
plt.plot([0.6,1.0],[0.6,1.0])

In [ ]:
mask = stats['sample_size'] == 10000
stats['model'][mask]

In [ ]:
plt.figure(figsize=(8,6))
for model in ['mae', 'dino', 'sup']:
    mask = (stats['model'] == model) & (stats['sample_size'] == 10000)
    plt.barh(
        stats['class_name'][mask], 
        stats['auc_max'][mask],
        label=model,
        alpha=0.5
    )
    
# plt.xscale('log')
# plt.xlim(80, 40000)
plt.legend()


In [ ]:
plt.figure(figsize=(18,8))
for sample_size in [100, 10000]:
    mask = (stats['model'] == 'sup') & (stats['sample_size'] == sample_size)
    plt.barh(
        stats['class_name'][mask], 
        stats['auc_max'][mask],
        label=sample_size,
        alpha=0.5
    )
    
# plt.xscale('log')
# plt.xlim(80, 40000)
plt.title('SUP AUC for different patch sizes')
plt.legend()

In [ ]:
stats['model']

In [ ]:
def plot_diff(sample_size):
    plt.figure(figsize=(8,6))
    xs = []
    ys = []
    for cls in classes:
        mask = {model: (stats['class_name'] == cls) & (stats['model'] == model) & (stats['sample_size'] == sample_size) for model in ['mae', 'dino', 'sup']}
        if mask['mae'].sum() == 0 or mask['dino'].sum() == 0  or mask['sup'].sum() == 0:
            continue

        xs.append(cls)
        diff = stats['auc_max'][mask['mae']][0] - stats['auc_max'][mask['sup']][0]
        ys.append(diff)

    xs = np.array(xs)
    ys = np.array(ys)
    s = np.argsort(ys)
    plt.barh(
        xs[s], ys[s],
    )

    # plt.xscale('log')
    # plt.xlim(80, 40000)
#     plt.legend()
    plt.title(f"AUC difference between MAE and SUP_VIT ({sample_size} samples)")


In [ ]:
plot_diff(100)

In [ ]:
plot_diff(10000)

In [ ]:
stats

In [ ]:
val_losses={}


for run_hash in tqdm(repo.list_active_runs()):
    try:
#         print(run_hash)
        run = Run(run_hash)
        if ("cs_256_mae_bilinear_upsampling" in run.experiment):
            for metric in run.metrics():
                if metric.name == 'val_loss':
                     print(metric.name)
                    val_losses[run.experiment] = metric.data
    except Exception as e:
        print(f"{run_hash}", e)
        

In [ ]:
repo.list_active_runs()

In [ ]:
run = Run('7ac78c9f263445b085170f94')

In [6]:
import os

In [12]:
len(os.listdir('/home/hkhachatrian/mae/.aim/meta/locks/'))

0

In [11]:
len(os.listdir('/home/hkhachatrian/mae/.aim/seqs/progress/'))

438